<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import csv
from keras import layers,Input
from keras.models import Model
from urllib.request import urlopen
from sklearn.cluster import KMeans
import pandas as pd

# Stage 1 資料前處理

## 讀入向量化資料


In [2]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
df = pd.DataFrame(data).T

## 讀入 True label csv

In [3]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/TrainLabel.csv'
response = urlopen(url)
lines = [l.decode('utf-8') for l in response.readlines()]
true_labels = list(csv.reader(lines))
del true_labels[0]

In [4]:
len(true_labels)

1381

## 建立 False label 資料

In [5]:
import random

input_keys = list(data.keys())
false_labels = []

while len(false_labels) < len(true_labels):
  temp = [random.choice(input_keys),random.choice(input_keys)]
  if temp not in true_labels:
    false_labels.append(temp)

In [6]:
len(false_labels)

1381

## 產生所需資料格式

In [7]:
def data_generator(input,Label=0):
  return [[[data[row[0]],data[row[1]]],Label] for row in input]

In [8]:
# 標籤為True的向量資料對
true_list = data_generator(true_labels,1)

In [9]:
# 標籤為False的向量資料對
false_list = data_generator(false_labels,0)

In [10]:
df_true = pd.DataFrame(true_list)
df_true.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [11]:
df_false = pd.DataFrame(false_list)
df_false.head()

,0,1
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 2,...",0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
4,"[[1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...",0


## 混合 Ture / False 標籤的資料

In [20]:
train_list = true_list+false_list

In [13]:
len(train_list)

2762

In [23]:
df_train = pd.DataFrame(train_list)
df_train.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [24]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)

In [25]:
df_train.head()

,0,1
744,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
2171,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2532,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
831,"[[0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
699,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [28]:
train_x = df_train[0]
train_y = df_train[1]

# 建立神經網路模型

參考資料：[Image similarity estimation using a Siamese Network with a triplet loss](https://keras.io/examples/vision/siamese_network/)

## 建構原理
藉由同一個層來一起進行萃取兩篇文章的相似度特徵  
因此需共享權重來訓練神經網路

In [ ]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None,include_top=False)

left_input = Input(shape=(250,250,3))
right_input = Input(shape=(250,250,3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

print(left_features.shape)
print(right_features.shape)

merged_features = layers.concatenate([left_features,right_features],axis=-1)